In [2]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm

df = pd.read_csv("하이닉스 power1_data.csv", thousands=',', encoding='utf-8')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_7_features = [feature for feature, importance in features_rf_sorted[:7]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_7_features]
y = df['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
# Define y as before
y = df['target']

# Precision 점수를 저장할 리스트 초기화
precision_scores = []

# tqdm을 사용하여 random_state에 대한 반복 실행
for state in tqdm(range(1, 101), desc='Evaluating Random States'):
    X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=state)
    
    svc_model = DecisionTreeClassifier(random_state=state)
    svc_model.fit(X_train, y_train)
    y_pred = svc_model.predict(X_test)
    
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    precision_scores.append({'Random_State': state, 'Precision': precision})

# 리스트를 DataFrame으로 변환
precision_scores_df = pd.DataFrame(precision_scores)

# DataFrame 출력 설정 변경
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 결과 출력
precision_scores_df_sorted = precision_scores_df.sort_values(by='Precision', ascending=False)
precision_scores_df_sorted

Evaluating Random States:   0%|          | 0/100 [00:00<?, ?it/s]

,Random_State,Precision
89,90,0.573205
43,44,0.567513
48,49,0.563380
42,43,0.551711
77,78,0.550172
14,15,0.549837
39,40,0.549388
36,37,0.549184
18,19,0.548501
57,58,0.540968


In [5]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm

df = pd.read_csv("하이닉스 power1_data.csv", thousands=',', encoding='utf-8')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_7_features = [feature for feature, importance in features_rf_sorted[:7]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_7_features]
y = df['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
# Define y as before
y = df['target']

# Precision 점수를 저장할 리스트 초기화
precision_scores = []

# tqdm을 사용하여 random_state에 대한 반복 실행
for state in tqdm(range(1, 101), desc='Evaluating Random States'):
    X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=state)
    
    svc_model = LogisticRegression(random_state=state)
    svc_model.fit(X_train, y_train)
    y_pred = svc_model.predict(X_test)
    
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    precision_scores.append({'Random_State': state, 'Precision': precision})

# 리스트를 DataFrame으로 변환
precision_scores_df = pd.DataFrame(precision_scores)

# DataFrame 출력 설정 변경
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 결과 출력
precision_scores_df_sorted = precision_scores_df.sort_values(by='Precision', ascending=False)
precision_scores_df_sorted

Evaluating Random States:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

,Random_State,Precision
54,55,0.676018
58,59,0.626054
95,96,0.619161
79,80,0.602453
30,31,0.590501
65,66,0.587507
28,29,0.583627
44,45,0.583198
72,73,0.582758
93,94,0.581886
